In [1]:
import numpy as np
import time
import scipy.io as scio
import pandas as pd
import h5py
import hdf5storage
import os
import glob
import pyreadr

In [ ]:
work in Dir /home/chenshengquan/program/clusterATAC/supplementary_code/data/mouse13/use_mouse13_peaks/renbing
wget http://hgdownload.cse.ucsc.edu/goldenpath/mm9/bigZips/
../twoBitToFa mm9.2bit mm9.fa
use pipeline in https://github.com/r3fang/snATAC
./bwa/bwa index mm9.fa
./bwa/bwa mem -t 64 mm9.fa p56.rep1.R1.decomplex.fastq.gz p56.rep1.R2.decomplex.fastq.gz | ~/anaconda2/bin/samtools view -bS - > new_p56.rep1.bam
~/anaconda2/bin/samtools sort -t 5 -@ 64 p56.rep1.bam -o p56.rep1.sort.bam
~/anaconda2/bin/samtools rmdup -s p56.rep1.sort.bam p56.rep1.sort.dedup.bam
~/anaconda2/bin/samtools index p56.rep1.sort.dedup.bam

In [4]:
cell_labels = hdf5storage.loadmat('../REN/original/cell_labels.mat')
cell_labels = cell_labels['cell_labels'].reshape(-1)
cell_labels.shape

(2088,)

In [5]:
cell_idx_sc = hdf5storage.loadmat('../REN/original/agg0.50_selectedSc.mat')
cell_idx_sc = cell_idx_sc['cell_idx_sc'].reshape(-1)
cell_idx_sc.shape

(1043,)

In [6]:
label_mat = hdf5storage.loadmat('../REN/original/agg0.50_cell_label.mat')['half_cell_label'].reshape(-1)
label_mat.shape
# same as cell_label in agg0.50.mat

(1043,)

In [7]:
np.sum(cell_labels[cell_idx_sc-1] == label_mat)

1043

In [8]:
cell_barcodes = scio.loadmat('../REN/original/cell_barcodes.mat')['cell_barcodes'].reshape(-1)
cell_barcodes.shape

(2088,)

In [9]:
cell_barcodes = np.array([i[0] for i in cell_barcodes])

In [10]:
half_sc_label = cell_labels[cell_idx_sc-1]
half_sc_barcode = cell_barcodes[cell_idx_sc-1]
print(half_sc_label.shape, half_sc_barcode.shape)

(1043,) (1043,)


In [11]:
cell_idx_bulk = np.setdiff1d(np.array(range(2088)), cell_idx_sc-1)
half_bulk_label = cell_labels[cell_idx_bulk]
half_bulk_barcode = cell_barcodes[cell_idx_bulk]
print(half_bulk_label.shape, half_bulk_barcode.shape)

(1045,) (1045,)


In [12]:
half_sc_label = np.array([i.strip() for i in half_sc_label])
half_bulk_label = np.array([i.strip() for i in half_bulk_label])

In [435]:
fout = open('../R1_data_raw/half_sc_meta.txt','w')
for i in range(half_sc_label.shape[0]):
    fout.write('%s\t%s\n'%(half_sc_barcode[i].split('_')[1],half_sc_label[i]))
fout.close()

In [436]:
fout = open('../R1_data_raw/half_bulk_meta.txt','w')
for i in range(half_bulk_label.shape[0]):
    fout.write('%s\t%s\n'%(half_bulk_barcode[i].split('_')[1],half_bulk_label[i]))
fout.close()

## select barcode

In [14]:
for method_name in ['RA3','scABC','SCALE','Scasat','SnapATAC','cisTopic','Cusanovich2018']:
    fout_bash = open('../R1_data_raw/script/%s.sh'%(method_name),'w')
    for exc_peak_num in [500]:
        if method_name == 'RA3':
            suffix = '_Bulkforebrain_LOMgOc'
            dim = '5'
        else:
            suffix = ''
            dim = '10'
        if method_name == 'scABC':
            dim = '999'
        expr_name = '%s_forebrain_half_peak300_LOspePeakMgOc_top%d_peak000_dim%s%s'%(method_name,exc_peak_num,dim,suffix)
        cluster_res = pd.read_csv('../R1_data_raw/clusters/%s_clusters.tsv'%expr_name,sep='\t',index_col=0)
        if method_name != 'scABC' and np.sum(cluster_res['label'].values == half_sc_label) != half_sc_label.shape[0]:
            print('LABEL ERROR: %s'%expr_name)
        try:
            cluster_label = cluster_res['louvain'].values
        except:
            cluster_label = cluster_res['scABC'].values
        
        for idx, cluster in enumerate(np.unique(cluster_label)):
            fout = open('../R1_data_raw/clusters/%s_cluster%d.txt'%(expr_name, idx),'w')
            c_barcode = half_sc_barcode[cluster_label == cluster]
            c_label = half_sc_label[cluster_label == cluster]
            for i in range(c_barcode.shape[0]):
                fout.write('%s\n'%(c_barcode[i].split('_')[1]))
#                 fout.write('%s\t%s\n'%(c_barcode[i].split('_')[1], c_label[i]))
            fout.close()
            fout_bash.write('~/anaconda2/bin/samtools view -@ 64 ~/program/clusterATAC/supplementary_code/data/mouse13/use_mouse13_peaks/renbing/p56.rep1rep2.sort.dedup.bam | LC_ALL=C grep -F -f ../clusters/%s_cluster%d.txt > ../cluster_bam/%s_cluster%d.body.sam\n'%(expr_name, idx, expr_name, idx))
            fout_bash.write('cat ~/program/clusterATAC/supplementary_code/data/mouse13/use_mouse13_peaks/renbing/p56.rep1rep2.sort.dedup.head ../cluster_bam/%s_cluster%d.body.sam > ../cluster_bam/%s_cluster%d.sam\n'%(expr_name, idx, expr_name, idx))
            fout_bash.write('~/anaconda2/bin/samtools view -b -@ 64 ../cluster_bam/%s_cluster%d.sam > ../cluster_bam/%s_cluster%d.bam\n'%(expr_name, idx, expr_name, idx))
            fout_bash.write('rm -f ../cluster_bam/%s_cluster%d.sam\n'%(expr_name, idx))
            fout_bash.write('~/anaconda2/bin/samtools index ../cluster_bam/%s_cluster%d.bam\n'%(expr_name, idx))

    fout_bash.close()

## filter bam file

In [ ]:
work in Dir /home/chenshengquan/program/clusterATAC/supplementary_code/data/mouse13/use_mouse13_peaks/renbing
# Save the header lines
~/anaconda2/bin/samtools view -H p56.rep1rep2.sort.dedup.bam > p56.rep1rep2.sort.dedup.head
exec scripts in ../R1_data_raw/script

## call peak

In [ ]:
work in /home/chenshengquan/program/clusterATAC/supplementary_code/data/R1_data_raw/script

In [18]:
for method_name in ['RA3','scABC','SCALE','Scasat','SnapATAC','cisTopic','Cusanovich2018']:
    fout_bash = open('../R1_data_raw/script/%s_callPeak.sh'%(method_name),'w')
    for exc_peak_num in [500]:
        if method_name == 'RA3':
            suffix = '_Bulkforebrain_LOMgOc'
            dim = '5'
        else:
            suffix = ''
            dim = '10'
        if method_name == 'scABC':
            dim = '999'
        expr_name = '%s_forebrain_half_peak300_LOspePeakMgOc_top%d_peak000_dim%s%s'%(method_name,exc_peak_num,dim,suffix)
        
        fout_bash.write('cd ../cluster_bam/\n')
        for idx in range(np.unique(cluster_label).shape[0]):
#             env: ~/anaconda2/bin
            fout_bash.write('~/program/clusterATAC/supplementary_code/data/mouse13/snATAC/bin/snATAC_pre -t 64 -m 30 -f 2000 -e 75 -i %s_cluster%d.bam -o ../peaks/%s_cluster%d.bed.gz 2> ../peaks/%s_cluster%d.pre.log\n'%(expr_name, idx, expr_name, idx, expr_name, idx))
#             ~/anaconda3/bin/macs2
            fout_bash.write('macs2 callpeak -t ../peaks/%s_cluster%d.bed.gz -f BED -n ../peaks/%s_cluster%d --outdir ../peaks/ -g mm -p 0.05 --nomodel --shift 150 --keep-dup all\n'%(expr_name, idx, expr_name, idx))
fout_bash.close()

In [ ]:
# ~/anaconda3/envs/R40/bin/R
library(Rsubread)
library(GenomicRanges)
for (method_name in c('RA3','scABC','SCALE','Scasat','SnapATAC','cisTopic','Cusanovich2018')){
    for (exc_peak_num in c(500)){
        if (method_name == 'RA3'){
            suffix = '_Bulkforebrain_LOMgOc'
            dim = '5'
        }
        else{
            suffix = ''
            dim = '10'
        }
        if (method_name == 'scABC'){
            dim = '999'
        }
        expr_name = sprintf('%s_forebrain_half_peak300_LOspePeakMgOc_top%d_peak000_dim%s%s',method_name,exc_peak_num,dim,suffix)
        
        for (idx in c(0,1,2,3,4,5,6,7)){
            print(sprintf("../peaks/%s_cluster%d_peaks.narrowPeak",expr_name,idx))
            peaks.df <- read.table(sprintf("../peaks/%s_cluster%d_peaks.narrowPeak",expr_name,idx))
            # remove top 5% peaks
            cutoff <- quantile((peaks.df$V5), probs = 0.95)
            peaks.df <- peaks.df[which(peaks.df$V5 < cutoff),]
            # proms.df <- read.table("mm10/mm10.refSeq_promoter.bed")
            proms.df <- promoterRegions(annotation="mm9")
            peaks.gr <- GRanges(peaks.df[,1], IRanges(peaks.df[,2], peaks.df[,3]))
            proms.gr <- GRanges(proms.df[,2], IRanges(proms.df[,3], proms.df[,3]))

            peaks.sel.gr <- peaks.gr[-queryHits(findOverlaps(peaks.gr, proms.gr))]
            peaks.sel.ex.gr <- resize(reduce(resize(peaks.sel.gr, 1000, 
                                      fix="center")), 1000, fix="center")

            peaks.sel.ex.df <- as.data.frame(peaks.sel.ex.gr)[,1:3]
            peaks.sel.ex.df <- peaks.sel.ex.df[which(peaks.sel.ex.df$start > 0),]
            peaks.sel.ex.df <- peaks.sel.ex.df[which(peaks.sel.ex.df$end > 0),]
            write.table(peaks.sel.ex.df, file = sprintf("../peaks/%s_cluster%d_peaks.filtered.bed",expr_name,idx), 
                        append = FALSE, quote = FALSE, sep = "\t", 
                        eol = "\n", na = "NA", dec = ".", 
                        row.names = FALSE, col.names = FALSE, 
                        qmethod = c("escape", "double"),
                        fileEncoding = "")
        }
    }
}

## merge peak

In [433]:
fout_bash = open('../R1_data_raw/script/mergePeak.sh','w')

for method_name in ['RA3','scABC','SCALE','Scasat','SnapATAC','cisTopic','Cusanovich2018']:
    for exc_peak_num in [500]:
        if method_name == 'RA3':
            suffix = '_Bulkforebrain_LOMgOc'
            dim = '5'
        else:
            suffix = ''
            dim = '10'
        if method_name == 'scABC':
            dim = '999'
        expr_name = '%s_forebrain_half_peak300_LOspePeakMgOc_top%d_peak000_dim%s%s'%(method_name,exc_peak_num,dim,suffix)
        
        fout_bash.write('cat ../peaks/%s_cluster?_peaks.filtered.bed > ../peaks/%s_allcluster_peaks.filtered.bed\n'%(expr_name,expr_name))
        fout_bash.write('sortBed -i ../peaks/%s_allcluster_peaks.filtered.bed > ../peaks/%s_allcluster_peaks.filtered.sorted.bed\n'%(expr_name,expr_name))
        fout_bash.write('bedtools merge -i ../peaks/%s_allcluster_peaks.filtered.sorted.bed > ../peaks/%s_allcluster_peaks.filtered.sorted.merged.bed\n'%(expr_name,expr_name))

fout_bash.close()

## count mtx

In [22]:
fout_bash = open('../R1_data_raw/script/countMtx.sh','w')
# sc_atac_window_counter False
for method_name in ['scABC','SCALE','Scasat','SnapATAC','cisTopic','Cusanovich2018']:
    for exc_peak_num in [500]:
        if method_name == 'RA3':
            suffix = '_Bulkforebrain_LOMgOc'
            dim = '5'
        else:
            suffix = ''
            dim = '10'
        if method_name == 'scABC':
            dim = '999'
        expr_name = '%s_forebrain_half_peak300_LOspePeakMgOc_top%d_peak000_dim%s%s'%(method_name,exc_peak_num,dim,suffix)
        fout_bash.write('~/anaconda2/bin/python ~/program/clusterATAC/supplementary_code/sc_atac_window_counter.py ~/program/clusterATAC/supplementary_code/data/mouse13/use_mouse13_peaks/renbing/p56.rep1rep2.sort.dedup.bam ../half_sc_meta.txt ../peaks/%s_allcluster_peaks.filtered.sorted.merged.bed ../mtx/%s.mat.txt False &\n'%(expr_name,expr_name))

fout_bash.close()

In [3]:
# sc_atac_window_counter True
for method_name in ['RA3']:
    for exc_peak_num in [500]:
        if method_name == 'RA3':
            suffix = '_Bulkforebrain_LOMgOc'
            dim = '5'
        else:
            suffix = ''
            dim = '10'
        if method_name == 'scABC':
            dim = '999'
        expr_name = '%s_forebrain_half_peak300_LOspePeakMgOc_top%d_peak000_dim%s%s'%(method_name,exc_peak_num,dim,suffix)
        print('~/anaconda2/bin/python ~/program/clusterATAC/supplementary_code/sc_atac_window_counter.py ~/program/clusterATAC/supplementary_code/data/mouse13/use_mouse13_peaks/renbing/p56.rep1rep2.sort.dedup.bam ../half_sc_meta.txt ../peaks/%s_allcluster_peaks.filtered.sorted.merged.bed ../mtx/%s.mat.txt True &\n'%(expr_name,expr_name))
        print('~/anaconda2/bin/python ~/program/clusterATAC/supplementary_code/sc_atac_window_counter.py ~/program/clusterATAC/supplementary_code/data/mouse13/use_mouse13_peaks/renbing/p56.rep1rep2.sort.dedup.bam ../half_bulk_meta.txt ../peaks/%s_allcluster_peaks.filtered.sorted.merged.bed ../mtx/%s.bulk.mat.txt True &\n'%(expr_name,expr_name))


## save mat

In [1]:
for method_name in ['RA3','scABC','SCALE','Scasat','SnapATAC','cisTopic','Cusanovich2018']:
    for exc_peak_num in [500]:
        if method_name == 'RA3':
            suffix = '_Bulkforebrain_LOMgOc'
            dim = '5'
        else:
            suffix = ''
            dim = '10'
        if method_name == 'scABC':
            dim = '999'
            
        expr_name = '%s_forebrain_half_peak300_LOspePeakMgOc_top%d_peak000_dim%s%s'%(method_name,exc_peak_num,dim,suffix)
        ren_df = pd.read_csv('../R1_data_raw/mtx/%s.mat.txt'%expr_name, sep='\t')
        ren_peaks = ren_df.iloc[:,:3].values
        ren_count = ren_df.iloc[:,4:].values
        ren_label = half_sc_label

        filter_peak = np.sum(ren_count >= 1, axis=1) >= round(ren_count.shape[1]*0.03)

        file_name = '%s_forebrain_half_peak300_LOspePeakMgOc_top%d_peak000_dim%s'%(method_name,exc_peak_num,dim)
        hdf5storage.savemat('../R1_data/%s.mat'%(file_name), {'count_mat':ren_count, 'label_mat':ren_label.reshape(-1,1)})

        fout = open('../R1_data/%s.txt'%(file_name),'w')
        for item in ren_peaks:
            fout.write('_'.join([item[0],str(item[1]),str(item[2])])+'\n')
        fout.close()

In [2]:
for exc_peak_num in [500]:
    method_name = 'RA3'
    suffix = '_Bulkforebrain_LOMgOc'
    dim = '5'

    expr_name = '%s_forebrain_half_peak300_LOspePeakMgOc_top%d_peak000_dim%s%s'%(method_name,exc_peak_num,dim,suffix)
    ren_df = pd.read_csv('../R1_data_raw/mtx/%s.bulk.mat.txt'%expr_name, sep='\t')
    ren_peaks = ren_df.iloc[:,:3].values
    ren_count = ren_df.iloc[:,4:].values
    ren_label = half_bulk_label
    filter_peak = np.sum(ren_count >= 1, axis=1) >= round(ren_count.shape[1]*0.03)
    
    selected_bulk_mat = ren_count
    selected_bulk_label = ren_label.reshape(-1,1)
    bulk_from_sc = np.zeros((selected_bulk_mat.shape[0],np.unique(selected_bulk_label).shape[0]))
    for i, celltype in enumerate(np.unique(selected_bulk_label)):
        bulk_from_sc[:,i] = np.mean(selected_bulk_mat[:,selected_bulk_label[:,0]==celltype], axis=1)
    
    file_name = '%s_forebrain_half_peak300_LOspePeakMgOc_top%d_peak000_dim%s'%(method_name,exc_peak_num,dim)
    hdf5storage.savemat('../R1_data/%s_bulk_mat.mat'%file_name, {'bulk_mat': bulk_from_sc})
